In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

epochs=40
batch_size=6
learning_rate=0.002

In [ ]:
train = datasets.MNIST('', train=True, download=True,transform=transforms.Compose([transforms.ToTensor()]))

test = datasets.MNIST('', train=False, download=True,transform=transforms.Compose([transforms.ToTensor()]))



In [ ]:
train_size = int(0.8 * len(train))          #spliting the training dataset
val_size = len(train) - train_size

train_set, val_set = torch.utils.data.random_split(train, [train_size, val_size])

train_load=torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)

val_load=torch.utils.data.DataLoader(val_set, batch_size=batch_size, shuffle=True)

test_load=torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=True)

In [ ]:
def imshow(img):
    img = img / 2 + 0.5  # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [ ]:
# get some random training images
dataiter = iter(train_load)
images, labels = next(dataiter)

# show images
imshow(torchvision.utils.make_grid(images))

In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(10, 20, 2)

        self.fc1 = nn.Linear(20 * 5 * 5, 50)
        self.fc2 = nn.Linear(50, 10)
    

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # -> n, 10, 14, 14
        x = self.pool(F.relu(self.conv2(x)))  # -> n, 20, 5, 5
        x = x.view(-1, 20 * 5 * 5)            # -> n, 400
        x = F.relu(self.fc1(x))               # -> n, 50
        x = F.softmax(self.fc2(x), dim=1)     # -> n, 10
        return x

    def convs(self, x):
        # max pooling over 2x2
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))

        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
        return x
    

In [ ]:
model = ConvNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
n_total_steps = len(train_load)
n_correct = 0
n_samples = 0

trn_loss = np.zeros([epochs,1])
trn_accuracy = np.zeros([epochs,1])
val_accuracy= np.zeros([epochs,1])
val_loss = np.zeros([epochs,1])


for epoch in range(epochs):
    loss_total=0
#Training
    for i, (images, labels) in enumerate(train_load):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        model.train()
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward
        loss.backward()
        optimizer.step()
        loss_total+=loss.item()
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

        if (i+1) % 2000 == 0:
            print (f'Epoch [{epoch+1}/{epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')   

    trn_loss[epoch]=loss_total/n_total_steps
    trn_accuracy[epoch] = 100.0 * n_correct / n_samples
    print(f'Loss after the Epoch {epoch+1}: {trn_loss[epoch]}')
    print(f'Accuracy of the Epoch {epoch+1}: {trn_accuracy[epoch]} %')


#validation
    for images, labels in val_load:
        images = images.to(device)
        labels = labels.to(device)
        model.eval()
        with torch.no_grad():
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            n_samples += labels.size(0)
            n_correct += (predicted == labels).sum().item()
            loss_v = criterion(outputs, labels)

    val_loss[epoch]=loss_v.item()
    val_accuracy[epoch] = 100.0 * n_correct / n_samples

print('Finished Training')
PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)

In [ ]:
# Training Accuracy and Loss
plt.rcParams["figure.figsize"] = (10,7)
plt.plot(trn_accuracy, label="Training")
plt.plot(val_accuracy, label= "Validation")
plt.xlabel('Iterations')
plt.legend()
plt.title('Accuracy Comparison')
plt.show()

In [ ]:
# Validation Accuracy and Loss
plt.plot(trn_loss, label="Training")
plt.plot(val_loss, label="Validation")
plt.xlabel('Iterations')
plt.legend()
plt.title('Loss over Epochs')
plt.show()

In [ ]:
#Testing
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_load:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')